# Private network - Storage account
This template sets up a private storage account with multiple private endpoint connections for table, queue and blob. Please ensure an Azure Private DNS Zone is created before proceeding with the deployment, see [/scenario-private-dns-zones.ipynb](../private-dns-zones/scenario-private-dns-zones.ipynb).

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed
- A private DNS zone, see [/scenario-private-dns-zones.ipynb](../scenario-private-dns-zones/private-dns-zones.ipynb)


## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [ ]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=northeurope
TEMPLATE_FILE = "main.bicep"
PARAMETERS_FILE = "main.bicepparam"

2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

deployment_name = "private-storageaccount-deployment"
template_file = os.getenv('TEMPLATE_FILE')
parameters_file = os.getenv('PARAMETERS_FILE')

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

4. Login and set default Azure subscription

In [ ]:
!az login
!az account set --subscription  {subscription_id}

5. Create an Azure Resource group:

In [ ]:
!az group create --name {resource_group} --location {location}

## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [ ]:
%%writefile {parameters_file}

using '#{template_file}#'

param storageAccountName = '${take('prvtstrg${uniqueString(vnetRsourceGroup)}', 15)}'
param vnetRsourceGroup = 'my-private-network'
param vnetName = 'private-network-test'
param subnetName = 'default'

2. Replace tokens in `.bicepparam` file

In [ ]:
%run "../../../utils/placeholder_replacer.py"

path = f'{parameters_file}'
replace_placeholders_in_file(path)

3. Deploy the template:

In [ ]:
!az deployment group create --resource-group {resource_group} --template-file {template_file}  --parameters {parameters_file} --name {deployment_name} 


4. Optional: Login to the bastion jumpbox and test the connection.

    - Login to the VM:
![bastion-login.png](../../../images/private-network-docs/bastion-login.png)
    - Get Storage account connection string:

In [ ]:
import os
import json

storage_account_resource_id = os.popen(f"az deployment group show --resource-group {resource_group} --name {deployment_name} --query \"properties.outputs.storageAccountResourceId.value\" --output tsv").read().strip()

print(f"Storage Account Id: {storage_account_resource_id}")

connection_string = json.loads(os.popen(f"az storage account show-connection-string --ids {storage_account_resource_id} --resource-group {resource_group}").read().strip())['connectionString']

print(f"Connection string: {connection_string}")

    - Connect in the VM with the Azure Storage Explorer to the storage account 
![connect-to-storage-account.png](../../../images/private-network-docs/connect-to-storage-account.png)
    
    - Explore! 
![storage-account-object-explorer.png](../../../images/private-network-docs/storage-account-object-explorer.png)